# Simple Impementation of E(n) Equivariant Graph Neural Networks

Original paper https://arxiv.org/pdf/2102.09844.pdf by Victor Garcia Satorras, Emiel Hoogeboom, Max Welling

In [ ]:
import time
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Load QM9 Dataset

In [ ]:
!git clone https://github.com/senya-ashukha/simple-equivariant-gnn.git
%cd simple-equivariant-gnn

Cloning into 'simple-equivariant-gnn'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 87 (delta 37), reused 31 (delta 5), pack-reused 0
Unpacking objects: 100% (87/87), done.
/content/simple-equivariant-gnn


In [ ]:
# QM9 is a dataset for Molecular Property Predictions http://quantum-machine.org/datasets/
# We will predict Highest occupied molecular orbital energy 
# https://en.wikipedia.org/wiki/HOMO_and_LUMO
# We use data loaders from the official repo

from qm9.data_utils import get_data, BatchGraph
train_loader, val_loader, test_loader, charge_scale = get_data(num_workers=1)

dict_keys([0, 1, 6, 7, 8, 9])
dict_keys([0, 1, 6, 7, 8, 9])
dict_keys([0, 1, 6, 7, 8, 9])


# Graph Representation

In [ ]:
batch = BatchGraph(iter(train_loader).next(), False, charge_scale)
batch

In the batch: num_graphs 96 num_nodes 1765
> .h 		 a tensor of nodes representations 		shape 1765 x 15
> .x 		 a tensor of nodes positions  			shape 1765 x 3
> .edges 	 a tensor of edges, a fully connected graph 	shape 31564 x 2
> .batch  	 a tensor of graph_ids for each node 		tensor([ 0,  0,  0,  ..., 95, 95, 95])

# Define Equivariant Graph Convs  & GNN

In [ ]:
def index_sum(agg_size, source, idx, cuda):
    """
        source is N x hid_dim [float]
        idx    is N           [int]
        
        Sums the rows source[.] with the same idx[.];
    """
    tmp = torch.zeros((agg_size, source.shape[1]))
    tmp = tmp.cuda() if cuda else tmp
    res = torch.index_add(tmp, 0, idx, source)
    return res

In [ ]:
class ConvEGNN(nn.Module):
    def __init__(self, in_dim, hid_dim, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        self.cuda = cuda
        
        # computes messages based on hidden representations -> [0, 1]
        self.f_e = nn.Sequential(nn.Linear(2 * in_dim + 1, hid_dim), nn.SiLU(),
                                 nn.Linear(hid_dim, hid_dim), nn.SiLU())
        
        # preducts "soft" edges based on messages 
        self.f_inf = nn.Sequential(nn.Linear(hid_dim, 1), nn.Sigmoid())
        
        # updates hidden representations -> [0, 1]
        self.f_h = nn.Sequential(nn.Linear(in_dim + hid_dim, hid_dim), nn.SiLU(), 
                                 nn.Linear(hid_dim, hid_dim))

        
    def forward(self, b):
        # compute distances for all edges
        edge_st, edge_end = b.edges[:,0], b.edges[:,1]
        dists = torch.sum((b.x[edge_end] - b.x[edge_st]) ** 2, dim=1).reshape(-1, 1)
        
        # compute messages
        tmp = torch.hstack([b.h[edge_st], b.h[edge_end], dists])
        m_ij = self.f_e(tmp)
        
        # predict edges
        e_ij = self.f_inf(m_ij)
        
        # average e_ij-weighted messages  
        # m_i is num_nodes x hid_dim
        m_i = index_sum(b.h.shape[0], e_ij*m_ij, b.edges[:,0], self.cuda)
        
        # update hidden representations
        b.h += self.f_h(torch.hstack([b.h, m_i]))

        return b

In [ ]:
class NetEGNN(nn.Module):
    def __init__(self, in_dim=15, hid_dim=128, out_dim=1, n_layers=7, cuda=True):
        super().__init__()
        self.hid_dim=hid_dim
        
        self.emb = nn.Linear(in_dim, hid_dim)

        # Make gnn of n_layers
        self.gnn = [ConvEGNN(hid_dim, hid_dim, cuda=cuda) for l in range(n_layers)]
        self.gnn = nn.Sequential(*self.gnn)
        
        self.pre_mlp = nn.Sequential(
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, hid_dim))
        
        self.post_mlp = nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(hid_dim, hid_dim), nn.SiLU(),
            nn.Linear(hid_dim, out_dim))

        if cuda: self.cuda()
        self.cuda = cuda
    
    def forward(self, b):
        b.h = self.emb(b.h)
        
        b = self.gnn(b)
        h_nodes = self.pre_mlp(b.h)
        
        # h_graph is num_graphs x hid_dim
        h_graph = index_sum(b.nG, h_nodes, b.batch, self.cuda) 
        
        out = self.post_mlp(h_graph)
        return out

In [ ]:
epochs = 200
cuda = True

model = NetEGNN(n_layers=7, cuda=cuda)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-16) # lr=5e-4
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, verbose=False)

# Training

In [ ]:
print('> start training')

tr_ys = train_loader.dataset.data['homo']
me, mad = torch.mean(tr_ys), torch.mean(torch.abs(tr_ys - torch.mean(tr_ys)))

if cuda:
    me = me.cuda()
    mad = mad.cuda()

train_loss = []
val_loss = []
test_loss = []

for epoch in range(epochs):
    print('> epoch %s:' % str(epoch).zfill(3), end=' ', flush=True) 
    start = time.time()

    batch_train_loss = []
    batch_val_loss = []
    batch_test_loss = []

    model.train()
    for batch in train_loader:
        batch = BatchGraph(batch, cuda, charge_scale)
        
        out = model(batch).reshape(-1)
        loss =  F.l1_loss(out, (batch.y-me)/mad)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        with torch.no_grad():
            loss =  F.l1_loss(out*mad+me, batch.y)

        batch_train_loss += [float(loss.data.cpu().numpy())]  
        
    train_loss += [np.mean(batch_train_loss)/0.001]
    
    print('train %.3f' % train_loss[-1], end=' ', flush=True)
    
    with torch.no_grad():
        model.eval()
        for batch in val_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_val_loss += [np.mean(loss)]
            
        val_loss += [np.mean(batch_val_loss)/0.001]
        
        print('val %.3f' % val_loss[-1], end=' ', flush=True)
        
        for batch in test_loader:
            batch = BatchGraph(batch, cuda, charge_scale)
            out = model(batch).reshape(-1)
            loss = F.l1_loss(out*mad+me, batch.y).data.cpu().numpy()
            batch_test_loss += [np.mean(loss)]

        test_loss += [np.mean(batch_test_loss)/0.001]
        
    end = time.time()

    print('test %.3f (%.1f sec)' % (test_loss[-1], end-start), flush=True)
    lr_scheduler.step()

> start training
> epoch 000: train 331.462 val 307.934 test 310.421 (85.8 sec)
> epoch 001: train 228.864 val 233.028 test 228.634 (82.7 sec)
> epoch 002: train 185.693 val 166.203 test 163.385 (82.9 sec)
> epoch 003: train 163.003 val 137.879 test 136.068 (82.6 sec)
> epoch 004: train 145.501 val 143.238 test 145.194 (82.7 sec)
> epoch 005: train 132.975 val 120.555 test 120.356 (82.7 sec)
> epoch 006: train 123.954 val 114.002 test 112.166 (82.5 sec)
> epoch 007: train 117.180 val 106.519 test 106.249 (83.0 sec)
> epoch 008: train 109.622 val 104.664 test 103.775 (82.4 sec)
> epoch 009: train 105.312 val 98.981 test 100.688 (82.5 sec)
> epoch 010: train 100.985 val 86.974 test 87.005 (82.4 sec)
> epoch 011: train 97.742 val 99.523 test 98.275 (82.7 sec)
> epoch 012: train 93.679 val 87.961 test 86.577 (81.9 sec)
> epoch 013: train 91.089 val 82.321 test 81.607 (82.7 sec)
> epoch 014: train 89.260 val 78.265 test 79.155 (82.3 sec)
> epoch 015: train 86.124 val 93.564 test 94.063 (81.

In [ ]:
print('Best result: ', test_loss[-1])
plt.plot(test_loss[::10])
plt.ylabel('Cost')
plt.xlabel('Iterations (by 10)')   
plt.show()